# Lab 04. Text Classification


This lab is devoted to text classification tasks.
- **Part 1 [4 points]**: you will be offered to separate Russian surnames from Russian words with Vowpal Wabbit.
- **Part 2 [11 points]** is about very common NLP problem - sentiment analysis.
- **Part 3 [5 points]** include tasks on POS tagging and WordEmbeddings.


#### Evaluation

Each task has its value, **20 points** in total. If you use some open-source code please make sure to include the url.

#### How to submit

- Name your file according to this convention: `lab04_GroupNo_Surname_Name.ipynb`. If you don't have group number, put `nan` instead.
- Attach it to an **email** with **topic** `lab04_GroupNo_Surname_Name.ipynb`
- Send it to `cosmic.research.ml@yandex.ru`


## Part 1 [4+ points]. Vowpal Wabbit vs. Russian Surnames

Vowpal Wabbit is machine learning model that is very efficient in text processing. You may read about it here:

- [Github repo](https://github.com/JohnLangford/vowpal_wabbit/wiki)
- [Tutorial VW](https://github.com/JohnLangford/vowpal_wabbit/wiki/Tutorial)
- [Data Format](https://github.com/JohnLangford/vowpal_wabbit/wiki/Input-format)
- [Command Line Parameters](https://github.com/JohnLangford/vowpal_wabbit/wiki/Command-line-arguments)

The task is to separate two sets: Russian surnames and common Russian words.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Read and prepare the dataset:

In [ ]:
def read_list_from_file(filename, prefix=""):
    res = []
    with open(prefix + filename, "r") as input_file:
        for line in input_file.readlines():
            res.append(line.strip())
    return res

In [ ]:
surnames = np.array(read_list_from_file("russian_surnames.txt"))
all_words = np.array(read_list_from_file("russian.txt"))

In [ ]:
print(surnames.shape, np.random.choice(surnames, 10), sep='\n')
print(all_words.shape, np.random.choice(all_words, 10), sep='\n')

In [ ]:
surnames_labels = np.ones_like(surnames, dtype=int)
allwords_labels = - np.ones_like(all_words, dtype=int)

X = np.concatenate([surnames, all_words])
y = np.concatenate([surnames_labels, allwords_labels])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, stratify=y, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

## On VW format

We will be using the following form of vw-strings:
`label | namespace_1 | namespace_2 | ... | namespace_k`

Label is either `1` or `-1` for the binary case.

Each namespace is a group of features in VW format, for example: 
- `a:1 b:2 c:1`
- `a b b c`
- `a:1 b:1 b:1 c:1`

All these records are equivalent to VW

There are ways to manually add weights, tags and names of namespaces, you may read about it here:
https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Input-format

## Feature Engineering

As VW uses bag of words representations, we will use bag of letter-tokens.

**Task 1.1 [1 point]**
- Implement `tokenize_word`
- Implement `get_token_features`
- Answer the question: why splitting the word into 3 or 4 letter tokens might be useful in this particular task?

In [ ]:
def tokenize_word(word, token_len=3):
    ''' Function that splits word into sequence of tokens
    Args:
        word (string): input word
        token_len (int): length of each token
    Returns:
        list(str): list of tokens 
    '''
    tokens_list = []
    # YOUR CODE HERE
    
    return tokens_list

assert tokenize_word("cybersnatch") == ['cyb', 'ybe', 'ber', 'ers', 'rsn', 'sna', 'nat', 'atc', 'tch'], "smth's wrong"
print("tokenize_word: seems legit")

In [ ]:
def get_token_features(word, token_sizes=[3]):
    ''' Function that converts a word into vw-format feature-string.
    This feature string consists of several namespaces (1 or more), 
    each namespace corresponds to one token_size.
    Args:
        word (string): input word
        token_sizes (list(int)): token lengths used for feature extraction
    Returns:
        str: feature-string like "namespace_1 | namespace_2 | ... | namespace_n"
    '''
    # YOUR CODE HERE

    return # YOUR CODE HERE

test_output_0 = get_token_features("cybersnatch", token_sizes=[3])
test_output_1 = get_token_features("cybersnatch", token_sizes=[2, 4])
assert test_output_0 == "cyb ybe ber ers rsn sna nat atc tch", "smth's wrong"
assert test_output_1 == "cy yb be er rs sn na at tc ch | cybe yber bers ersn rsna snat natc atch", "smth's wrong"
print(test_output_0)
print(test_output_1)
print("get_token_features: seems legit")

Don't skip the question part: *Answer the question: why splitting the word into 3 or 4 letter tokens might be useful in this particular task?*

**Your answer here**:

What you have just implemented (`get_token_features`) is an example of feature extractor. 
It takes an input word and returns a string that VW can process. 

As you may have noticed from VW data format, if you concatenate two feature strings (with ` | ` in the middle), you will also get a feature-string. 
We will use this fact while processing the whole dataset in vw-files.

**Task 1.2 [1 point]** Implement a function, takes a word, its label, list of feature-extractors and makes a labelled vw-string.

In [ ]:
def get_vw_line(word, label, feature_extractors):
    ''' Function that cooks a full vw-format string: "label | features".
    Features are extracted with `feature_extractors`. 
    Result of each feature_extractor is stored in separate namespace
    Args:
        word (string): input word
        label (int): its label
        feature_extractors (list(function)): list if feature_extractor functions
    Returns:
        str: string like "label | namespace_1 | namespace_2 | ... | namespace_n"
    '''
    # YOUR CODE HERE

    return # YOUR CODE HERE

Define some dummy extractors for the test:

In [ ]:
def dummy_tokenizer_1(word):
    return word[0]

def dummy_tokenizer_2(word):
    return word[::-1]

def dummy_tokenizer_3(word):
    return get_token_features(word, [2])

test_output = get_vw_line("test", -1, [dummy_tokenizer_1, dummy_tokenizer_2, dummy_tokenizer_3])
assert test_output == "-1 | t | tset | te es st", "smth's wrong"
print(test_output)
print("get_vw_line: seems legit")

Okay, now we are ready to prepare for the training: we will use 3 and 4 letter tokens as features:

In [ ]:
def simple_tokenizer(word):
    return get_token_features(word, [3, 4])

get_vw_line("cybersnatch", 1, [simple_tokenizer])

## Prepare input files

In [ ]:
def write_vw_file(words, labels, out_filename, feature_extractors):
    with open(out_filename, "w", encoding="utf8") as out_file:
        for word, label in zip(words, labels):
            line = get_vw_line(word, label, feature_extractors)
            out_file.write(line + "\n")

Create two files: one with train set, one with test set. It may take some time. Estimated size of train file is 200-300Mb

In [ ]:
write_vw_file(X_train, y_train, "vw_train", [simple_tokenizer])
write_vw_file(X_test, y_test, "vw_test", [simple_tokenizer])

Now it is time to install the mighty Vowpal Wabbit itself.

We suggest using command line version. It is installed with the following sequence of command line commands:
```
git clone --recursive https://github.com/VowpalWabbit/vowpal_wabbit.git 
cd vowpal_wabbit/; make 
cd vowpal_wabbit/; make install 
```
You may run them in jupyter terminal: use `New -> Terminal` on jupyter homepage (like here http://localhost:8888/tree/JupyterNotebooks or whatever port you are using).


Or just run the following code in a jupyter cell of this notebook:
```
%%capture
!git clone --recursive https://github.com/VowpalWabbit/vowpal_wabbit.git 
!cd vowpal_wabbit/; make 
!cd vowpal_wabbit/; make install 
```

In [ ]:
# %%capture
# !git clone --recursive https://github.com/VowpalWabbit/vowpal_wabbit.git 
# !cd vowpal_wabbit/; make 
# !cd vowpal_wabbit/; make install 

## Run the model

Run the following command to train a model.
- `-d vw_train` it learns on file "vw_train"
- `--loss_function=logistic` it optimizes logloss while training
- `-f model_v0.vw` it save the model into "model_v0.vw" file
- `--quiet` it does not report anything while training

In [ ]:
!vw -d vw_train --loss_function=logistic -f model_v0.vw --quiet

Now collect prediction:
- `-i model_v0.vw` - this model is used for predictions
- `-t vw_test` - these are objects to predict on
- `-p v0_predictions` - predictions are stored here

In [ ]:
!vw -i model_v0.vw -t vw_test -p v0_predictions --quiet 

In [ ]:
def read_predictions(filename):
    with open(filename, "r") as pred_file:
        predictions = [float(label) for label in pred_file.readlines()]
    return np.array(predictions)

Since vowpal wabbit predicts some scores, we will transform them into [0, 1] floats with sigmoid function:

In [ ]:
from scipy.stats import logistic

In [ ]:
pred_proba = logistic.cdf(read_predictions("v0_predictions"))
preds = np.array(pred_proba > 0.5, dtype=int) * 2 - 1

Let's measure quality of classification:

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score

In [ ]:
print("ROC AUC {:.5f}".format(roc_auc_score(y_test, pred_proba)))
print("Accuracy {:.5f}".format(accuracy_score(y_test, preds)))

You should get ROC AUC around 0.99 and accuracy around 0.95.
Not bad, right?

**Task 1.3** Improve accuracy with the following steps.

**Task 1.3.1 [0.5 points]**
Use feature ngrams while training (`--ngram`). Did they improve the score? Why?

Let's take a look at this example: 
- `ivanov -> iva van ano nov | ivan vano anov`
- It ends with `nov` which is a very common Russian surname ending.
- However, there are words like `november`, `innovation` and many others that have the same token `nov`.

**Task 1.3.2 [0.5 points]** Suggest and implement a method that will allow to separate tokens that are in the end of the word and that are not.

You will have to create a new `feature_extractor` that uses this method, rewrite train- and test- files and rerun training and testing parts.

Do we see any improvement?

**Task 1.3.3 [1 point]** Dataset contains surnames in different forms: `иванову, ивановой, ивановых` and so on.

Sometimes it is useful to normalize them before tokenization. You may use some external libs to convert every word to its normal form.
For example, you may use this one:
```
from pymorphy2 import MorphAnalyzer
```
You will need to update your list of feature_extractors and rewrite train- and test- files again.

**Task 1.3.4 [extra points]**

This is a place for your further experiments. Maximize Accuracy of the model.
- **[few extra points]** use new training parameters (may the `!vw --help` help you)
- **[many extra points]** invent new features 

In [ ]:
# !vw --help

## Part 2. Bag of Words vs. Bag of Popcorn [11 points]

This task is based on [Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data) competition. The goal is to label film reviews as positive or negative. 

Reviews may look like this:

```
I dont know why people think this is such a bad movie. Its got a pretty good plot, some good action, and the change of location for Harry does not hurt either. Sure some of its offensive and gratuitous but this is not the only movie like that. Eastwood is in good form as Dirty Harry, and I liked Pat Hingle in this movie as the small town cop. If you liked DIRTY HARRY, then you should see this one, its a lot better than THE DEAD POOL. 4/5
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk

In [ ]:
reviews = pd.read_csv("reviews.tsv", sep="\t")
reviews.head(3)

In [ ]:
X = reviews["review"]
y = reviews["sentiment"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=5000, random_state=42, stratify=y)

### Time to extract features

In this part of the assignment we will apply several methods of feature extraction and comapre them.

**Task 2.1 [0.5 point] - Simple BOW** 

In this task we will build a simple bow representation - without any preprocessing. 

For this purpose we will use [*CountVectorizer*](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) - a method that transforms text dataset into a [sparse matrix](https://docs.scipy.org/doc/scipy/reference/sparse.html).

Import CountVectorizer:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

Now try each of these approaches:
- fit vectorizer on X_train, apply to X_train, X_test
- fit vectorizer on X_train, apply to X_train; fit on X_test, apply to X_test
- fit vectorizer on X, apply to X_train, X_test

Report output matrix sizes in each case. 
- What is the difference? 
- Which of these approaches is the most fair and correct?

Use the most fair and correct one to get `X_train_0` and `X_test_0` - they will be needed for further tasks.

In [ ]:
count_vectorizer = CountVectorizer()

# YOUR CODE HERE

print(X_train_0.shape, X_test_0.shape)

In [ ]:
count_vectorizer = CountVectorizer()

# YOUR CODE HERE

print(X_train_0.shape, X_test_0.shape)

In [ ]:
count_vectorizer = CountVectorizer()

# YOUR CODE HERE

print(X_train_0.shape, X_test_0.shape)

**Task 2.2 [1 point] - S___se matrices**

What is the data type of `X_train_0` and `X_test_0`? What are those?

What differs them from usual np.arrays? Name several types how those special matrices are stored and what they are good for.

*Answer:*

**Task 2.3 [1.5 points] - Training**

Train LogisticRegression and Random forest on this data representations.
- Compare training time
- Compare Accuracy, precision, recall
- Plot ROC Curve and calculate ROC AUC (don't forget to predict_proba)
- Plot Precision-Recall curve and calculate f1-score (for example, with `plt.subplots(nrows=1, ncols=2)`)
- Print the trickiest missclassified objects. Why they were hard to classify?


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import time as tm

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, precision_recall_curve, roc_curve, roc_auc_score

In [ ]:
rf_model = RandomForestClassifier(n_estimators=500)
lr_model = LogisticRegression(max_iter=1e5)

Which model gives higher scores?

*Answer:*

### More sophisticated feature prerocessing

As we have seen, simple BOW can give us some result - it's time to improve it.

**Task 2.4 [1 point] - Frequencies calculation**

- Calculate top-20 words in train set and test set. *Are they meaningful?*
- Import `stopwords` and print some of them. What are those?
- Recalculate top-20 words in each set, but exclude stop words.
- Does now top-20 include more useful words?

In [ ]:
from collections import Counter
from nltk.tokenize import WhitespaceTokenizer, WordPunctTokenizer, TreebankWordTokenizer
from nltk.corpus import stopwords

**Task 2.5 [1 point] - Word Freqs by class**

How do you think, will top100 tokens for positive and negative classes be different? Use data to prove your point.

*Answer:*

**Task 2.6 [2 points] - Reducing dimensionality**

The goal is to reduce number of features to 15000.

Implement the following methods of dimensinality reduction:
1. Use CountVectorizer, but leave only 15k most frequent tokens
2. Use HashingVectorizer with 15k features
3. Use 15k most important features from perspective of previously trained RandomForest

*Hints:*
- in 1 and 2 you don't have to apply nltk.corpus.stopwords, vectorizers have `stopwords` parameter
- in 1 look for `vocabulary` parameter
- in 3... remember `lab02`? You may use `X_train_0` and `X_test_0` as input matrices

Train LogisticRegression and RandomForest on each dataset and compare ROC AUC scores of the classifiers.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer

**Task 2.7 [2 points] - Token Normalization**

Choose the best working method from previous task. Try improve it by applying a token normalization technique.

You may use one of normalizers imported below, but feel free to experiment.

Do the following:
- Apply normalizer to X_train, X_test
- Build BOW with CountVectorizer + stopwords. What are the shapes of train and test matrices now?
- Reduce dimensionality with the best method from Task 2.6. You may try all of them
- Train LR/RF to examine whether ROC AUC or Accuracy was improved.

In [ ]:
from nltk.stem import WordNetLemmatizer, PorterStemmer

**Task 2.8 [2 points] - Vowpal Wabbit**

An experiment! What if we omit all this manual feature engineering part and apply good all VW.

Prepare train and test files in the following format - without any preprocessing.
```
1 | The film was very very good, I liked ...
-1 | The worst film ever. Oh God ...
...
```

Train VW with not very sophisticated parameters, compare quality to LR/RF trained on datasets above. 

## Part 3. Word Embeddings [5 points]

In [ ]:
import gensim.downloader

Here is the list of pretrained word embedding models. We suggest using `glove-wiki-gigaword-100`.

In [ ]:
list(gensim.downloader.info()['models'].keys())

In [ ]:
word_embeddings = gensim.downloader.load("glove-wiki-gigaword-100")

**Task 3.1 [1 point] - WordEmbeddings Geometry**

As you probably know, vector space of word embeddings has non-trivial geometry: some word relations (like country-capital or single-plural) cab be represented by vectors, like: **(king - man) + woman = queen**

<img src="https://linkme.ufanet.ru/images/5687a2011b49eb2413912f1c7d0fb0bd.png" width=600px>

Check this statement on words from the above picture with `word_embeddings.most_similar` function. Pay attention to `positive` and `negative` params.

Provide **several** examples, make sure to present different relations: some for nouns, some for verbs.

**Task 3.2 [1 point] - POS analysis**

Use POS tagger to calculate most common POS in the dataset. 
Here you may read about nltk-taggers: [link](https://www.inf.ed.ac.uk/teaching/courses/icl/nltk/tagging.pdf)

- If you were to design POS-related weights, how would you do it?
- What POS would get the higher weight?

**Task 3.3 [3 points] - WordEmbeddings**

Use dense vector representations to construct vector-representation of each review, then train a model (LR or RF).

Compare results of the new model to results of the models above.
**Important**
- If you just sum embeddings of each token to get an embedding of the whole review, the cost of the task is **[2 points]**
- For **[3 points]** you have to use either TF-IDF weight or weights that you designed from POS tags.